In [0]:
import os
import numpy as np
import string
import nltk
from collections import defaultdict
nltk.download("stopwords")
from nltk.corpus import stopwords
import json
#create a dictionary of spam and ham that keeps the count of each words in the spam and ham traing dataset
#creates a set vocabulary that contains the unique words in the training set
def createDict():
  spam_train = os.listdir('drive/My Drive/Colab Notebooks/Dataset/train/spam/')
  spam_test = os.listdir('/content/drive/My Drive/Colab Notebooks/Dataset/test/spam/')
  ham_train = os.listdir('/content/drive/My Drive/Colab Notebooks/Dataset/train/ham/')
  ham_test = os.listdir('/content/drive/My Drive/Colab Notebooks/Dataset/test/ham/')
  vocabulary= set()
  # special_characters = "!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
  spam = defaultdict(int)
  ham = defaultdict(int)
  count_spam = 0
  count_ham = 0
  result = []
  count_spamfiles = 0
  count_hamfiles = 0
  for file in spam_train:
      count_spamfiles += 1
      f = open('/content/drive/My Drive/Colab Notebooks/Dataset/train/spam/'+file, errors='ignore')
      nopunc = ""
      line = f.read().replace('\n', ' ')
      for char in line:
          if char not in string.punctuation:
              nopunc += char

      # print(nopunc)
      words = nopunc.split()
      # clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
      for word in words:
        if word not in stopwords.words('english'):
          spam[word] += 1
          vocabulary.add(word)
          count_spam += 1

  for file in ham_train:
      count_hamfiles += 1
      f = open('/content/drive/My Drive/Colab Notebooks/Dataset/train/ham/'+file, 'r')
      nopunc = ""
      line = f.read().replace('\n', ' ')
      for char in line:
          if char not in string.punctuation:
              nopunc += char
        
      words = nopunc.split()        
      for word in words:
        if word not in stopwords.words('english'):
          ham[word] += 1
          vocabulary.add(word)
          count_ham += 1


  print("voc:",len(vocabulary))
  print("count of spam", count_spam)
  print("count of ham",count_ham)
  print(count_hamfiles)
  print(count_spamfiles)
  # return count_spam, count_ham, count_spamfiles, count_hamfiles, len(vocabulary),spam,ham
  dict = {"spamwords":spam,
          "hamwords":ham,
          "spam_count":count_spam,
          "ham_count":count_ham,
          "no_unique_words": len(vocabulary)       
          }
  with open('data1.json', 'w') as outfile:
      json.dump(dict, outfile)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
voc: 10274
count of spam 14354
count of ham 34846
340
133


In [0]:
#calculate the probability of each word in the tarining dataset
def calculateProbability(word, word_dict, b_c):
    ans = (word_dict[word]+1)/(b_c)
    return ans**(1./10)

In [0]:
#test the spam and ham test dataset
def testing(count_spam,count_ham, P_spam,P_ham,spam,ham, vocabulary):
  result = []
  ctspam = 0
  ctham = 0
  ham_test = os.listdir('/content/drive/My Drive/Colab Notebooks/Dataset/test/ham/')
  k = 1
  for file in ham_test:
    P1 = P_spam
    P2 = P_ham
    f = open('drive/My Drive/Colab Notebooks/Dataset/test/ham/'+file, errors='ignore')

    clean_words = f.read().replace('\n', ' ').split()
    for word in clean_words:
      P1 *= calculateProbability(word, spam, vocabulary+count_spam)
      P2 *= calculateProbability(word, ham, count_ham+vocabulary)

    if P1 >= P2:
        result.append('spam')
        ctspam += 1
    else:
        result.append('ham')
        ctham += 1
    print(" \r Processing", k ,"file out of", len(ham_test), end = " ")
    k+=1
  spam_test = os.listdir('/content/drive/My Drive/Colab Notebooks/Dataset/test/spam/')
  k = 1
  result = []
  for file in spam_test:
    P1 = P_spam
    P2 = P_ham
    f = open('drive/My Drive/Colab Notebooks/Dataset/test/spam/'+file, errors='ignore')

    clean_words = f.read().replace('\n', ' ').split()
    for word in clean_words:
      P1 *= calculateProbability(word, spam, vocabulary+count_spam)
      P2 *= calculateProbability(word, ham, count_ham+vocabulary)

    if P1 >= P2:
        result.append('spam')
        ctspam1 += 1
    else:
        result.append('ham')
        ctham1 += 1
    print(" \r Processing", k ,"file out of", len(ham_test), end = " ")
    k+=1
  return ctspam, ctham, ctspam1, ctham1

In [0]:
# calculate accuracy
def calculateAccuracy(ctspam, ctham, ctspam1, ctham1):
  correct_classification = ctspam1 + ctham
  accuracy = correct_classification/(ctspam + ctham + ctspam1 + ctham1)
  return accuracy

In [0]:
count_spam, count_ham, count_spamfiles, count_hamfile, no_unique_words,spam,ham = createdict()
P_spam = count_spamfiles/(count_spamfiles + count_hamfile)
P_ham = count_hamfile/(count_spamfiles + count_hamfile)
print(P_spam)
print(P_ham)
ctspam, ctham, ctspam1, ctham1 = testing(count_spam, count_ham, P_spam, P_ham, spam, ham, no_unique_words)
accuracy = calculateAccuracy(ctspam, ctham, ctspam1, ctham1)


